In [9]:
from conn import create_connection, read_into_df
import pandas as pd

conn = create_connection()

query = """
select cmr.map_id, crk.round_id, cmr.round_number, crk.killer_steam_id, crk.victim_steam_id, crk.time/1000000000 as seconds_elapsed
from historical_csgo.csgo_round_kills crk
left join historical_csgo.csgo_map_rounds cmr ON cmr.round_id = crk.round_id
where cmr.map_id in (
 SELECT map_id
    FROM historical_csgo.csgo_match_maps
    WHERE map_id NOT IN (
        SELECT DISTINCT(map_id)
        FROM historical_csgo.csgo_map_rounds cmr
        WHERE round_number != "t_score" + "ct_score"
        ORDER BY 1
    )
      AND map_id NOT IN (
        SELECT DISTINCT(map_id)
        FROM historical_csgo.csgo_map_rounds cmr
        GROUP BY 1
        HAVING MAX(round_number) < 16
        ORDER BY 1
    )
      AND map_id NOT IN (
        SELECT DISTINCT(map_id)
        FROM historical_csgo.csgo_match_maps cmm
        WHERE cmm.winner IS NULL
           OR cmm.winner = ''
    )
      AND match_id NOT IN (
        SELECT match_id
        FROM historical_csgo.csgo_match_data cmd
        WHERE cmd.date < {date}
    )
)
order by cmr.map_id, cmr.round_number, crk.time asc;
""".format(date="'2021-05-01'")

df = pd.read_sql(query, conn)
df.head()

/Users/matias/.local/share/virtualenvs/ModelNotebooks-OWhv5NqQ/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,map_id,round_id,round_number,killer_steam_id,victim_steam_id,seconds_elapsed
0,2,24,1,76561198164970560,76561198013243326,108
1,2,24,1,76561198114929868,76561197996370184,110
2,2,24,1,76561198164970560,76561198114929868,113
3,2,24,1,76561197960710573,76561198164970560,126
4,2,24,1,76561198016432560,76561197965021087,132


In [10]:
df.to_csv("../output/trades.csv", index=False)